In [1]:
import os
import sys
import pydicom
import numpy as np
import cv2

print('Python: {}'.format(sys.version))
print('Pydicom: {}'.format(pydicom.__version__))
print('Numpy: {}'.format(np.__version__))
print('CV2: {}'.format(cv2.__version__))

Python: 3.6.8 |Anaconda, Inc.| (default, Feb 21 2019, 18:30:04) [MSC v.1916 64 bit (AMD64)]
Pydicom: 2.2.1
Numpy: 1.19.5
CV2: 4.2.0


In [2]:
############### PARAMETERS ######################
IN_PATH = 'E:/data/CiM/'                      # path to dicoms exported from osirix
OUT_PATH = 'E:/data/CIM Ready/'               # path to save CIM formatted dicoms
size = (512,512)                              # desired image size
frames = 10                                   # number of temporal positions in each series

In [3]:
def convert_to_uint16(array):
    
    """
    Converts a numpy array into unsigned 16 bit integers
    inputs: array (numpy array)
    returns: 16-bit array
    """
    
    array = array.astype(np.float)
    array = array + np.abs(np.min(array))
    
    return array.astype(np.uint16)

In [4]:
def crop_and_resize(array, size):
    
    """
    Performs a center crop to square, then resizes to the specified size. 
    Inputs: array - the input image (numpy array of size (n,m))
            size - output image size (int, int)
            
    Returns: numpy array - the resized image 
             scale factor - the factor by which the image was scaled (used to update pixel spacing)  
    """
    
    try:
        height, width = array.shape
    except:
        print('Incorrect image dimension, please pass array of size NxM')
    
    
    if height > width:
        crop = int((height - width)/2)
        
        # crop the image to square, in the center
        cropped_array = array[crop:-crop, :]
        
    elif width > height:
        crop = int((width - height)/2)
        
        # crop the image to square, in the center
        cropped_array = array[:,crop:-crop]
        
    else:
        cropped_array = array #image already square
    
    # resize the image to the desired output size
    resized = cv2.resize(cropped_array, size, cv2.INTER_CUBIC)
    
    # store scaling factor
    scaling_factor = cropped_array.shape[0] / 512
    
    return resized, scaling_factor

In [5]:
def ct2mri_SA(input_path, output_path, frames=10, size=(512,512)):
    
    """
    Formats short axis CT images generated with Osirix for use with CIM 
    (ie, converts images from one time point per series, to one location per series)
    Inputs: input_path - the path to the short axis images (str)
            output_path - the path to save reformatted dicoms (str)
            frames - the number of frames (phases) per series (int)
            size - desired image size (tuple)
    
    Returns: None
    """

    if not os.path.isdir(output_path):
        os.mkdir(output_path)

    # iterate through directory, storing all files
    fileList = []
    timeList = []
    for (root, subdir, files) in os.walk(input_path):
        for file in files:
            fileList.append(os.path.join(root,file))
            timeList.append(file.split('-')[1])

    unique_frames = set(timeList)
    locations = int(len(fileList) / len(unique_frames))

    # record unique series instance uIDs
    seriesInstanceIDs = []
    for file in fileList:
        ds = pydicom.dcmread(file)
        seriesInstanceIDs.append(ds.SeriesInstanceUID)
    
    siUIDList = list(set(seriesInstanceIDs))
    
    patientID = fileList[0].split('/')[-1].split('\\')[0]

    # make patient specific output directory
    out_dir = output_path + '/{}/CT2MRI_F1_R10/'.format(patientID)

    if not os.path.isdir(out_dir):
        os.mkdir(out_dir)

    instance = 0
    trigger_time = 0

    # iterate over each phase
    for time in sorted(unique_frames):

        # grab all dicom files for this time point
        loc_files = [file for file in fileList if time in file.split('\\')[-1].split('-')[1]]

        # iterate through the unique locations
        for j in range(locations):

            # make output directory for this location
            if not os.path.isdir(out_dir +'/S{}/'.format(j)):
                os.mkdir(out_dir +'/S{}/'.format(j))

            # open the dicom and perform necessary formatting
            file = loc_files[j]
            ds = pydicom.dcmread(file)

            # format array
            image = ds.pixel_array
            image = convert_to_uint16(image)
            #image, scale = crop_and_resize(image, size)

            ds.PixelData = image.tobytes()
            ds.Rows, ds.Columns = image.shape

            # update header info
            #pixel_spacing = ds.PixelSpacing
            #new_pixel_spacing = [x * scale for x in pixel_spacing]

            #ds.PixelSpacing = new_pixel_spacing
            ds.InstanceNumber = instance
            ds.RescaleIntercept = np.min(image)
            ds.PixelRepresentation = 0
            ds.SeriesNumber = j
            ds.SeriesInstanceUID = siUIDList[0] + '.{}'.format(j)
            ds.WindowCenter = 2400
            ds.WindowWidth = 1000
            
            spacing = ds.SliceThickness
            ds.SpacingBetweenSlices = -spacing
            
            # add header info
            ds.add_new([0x0020,0x1041], 'DS', 0.0)
            ds.add_new([0x0020, 0x0100], 'IS', 1)
            ds.add_new([0x0020, 0x0105], 'IS', 1)
            ds.add_new([0x0018, 0x1081], 'IS', 754)
            ds.add_new([0x0018, 0x1082], 'IS', 1158)
            ds.add_new([0x0018, 0x9073], 'FD', 141.21)
            ds.add_new([0x0018, 0x0084], 'DS', 63.887)
            ds.add_new([0x0018, 0x1060], 'DS', trigger_time)
            
            try:
                # delete header info
                del ds[0x0008, 0x2142]
                del ds[0x0008, 0x2143]
                del ds[0x0008, 0x2144]
                del ds[0x0018, 0x0040]
            except:
                pass

            # save dicom to new folder
            new_path = out_dir + '/S{}/'.format(j) + file.split('\\')[-1]
            #print('Saving {}'.format(new_path))
            ds.save_as(new_path)

        # update instance number
        instance = instance + 1
        trigger_time += 32.0

In [6]:
def ct2mri_LA(input_path, output_path, frames=10, size=(512,512)):
    
    """
    Formats short axis CT images generated with Osirix for use with CIM 
    (ie, converts images from one time point per series, to one location per series)
    Inputs: input_path - the path to the short axis images (str)
            output_path - the path to save reformatted dicoms (str)
            frames - the number of frames (phases) per series (int)
            size - desired image size (tuple)
    
    Returns: None
    """

    if not os.path.isdir(output_path):
        os.mkdir(output_path)

    # iterate through directory, storing all files
    fileList = []
    timeList = []
    for (root, subdir, files) in os.walk(input_path):
        for file in files:
            fileList.append(os.path.join(root,file))

    # record unique series instance uIDs
    seriesInstanceIDs = []
    for file in fileList:
        ds = pydicom.dcmread(file)
        seriesInstanceIDs.append(ds.SeriesInstanceUID)

    siUIDList = list(set(seriesInstanceIDs))

    patientID = fileList[0].split('/')[-1].split('\\')[0]
    views = [x.split('\\')[-2] for x in fileList]
    viewsList = list(set(views))

    # make patient specific output directory
    out_dir = output_path + '/{}/CT2MRI_F1_R10/'.format(patientID)

    if not os.path.isdir(out_dir):
        os.mkdir(out_dir)


    # iterate over each view
    for j, view in enumerate(viewsList):
        
        instance = 0
        trigger_time = 0.0

        # grab all dicom files for this time point
        loc_files = [file for file in fileList if view in file]

        # iterate through the unique locations
        for file in loc_files:

            # make output directory for this location
            if not os.path.isdir(out_dir +'/{}/'.format(view)):
                os.mkdir(out_dir +'/{}/'.format(view))

            # open the dicom and perform necessary formatting
            ds = pydicom.dcmread(file)

            # format array
            image = ds.pixel_array
            image = convert_to_uint16(image)
            #image, scale = crop_and_resize(image, size)

            ds.PixelData = image.tobytes()
            ds.Rows, ds.Columns = image.shape

            # update header info
            #pixel_spacing = ds.PixelSpacing
            #new_pixel_spacing = [x * scale for x in pixel_spacing]

            #ds.PixelSpacing = new_pixel_spacing
            
            # update instance number
            instance = instance + 1
            ds.InstanceNumber = instance
            
            ds.RescaleIntercept = np.min(image)
            ds.PixelRepresentation = 0
            ds.SeriesNumber = j
            ds.SeriesInstanceUID = siUIDList[0] + '.{}'.format(j)
            ds.WindowCenter = 2400
            ds.WindowWidth = 1000
            
            spacing = ds.SliceThickness
            ds.SpacingBetweenSlices = -spacing
            
            # add header info
            ds.add_new([0x0020,0x1041], 'DS', 0.0)
            ds.add_new([0x0020, 0x0100], 'IS', 1)
            ds.add_new([0x0020, 0x0105], 'IS', 1)
            ds.add_new([0x0018, 0x1081], 'IS', 754)
            ds.add_new([0x0018, 0x1082], 'IS', 1158)
            ds.add_new([0x0018, 0x9073], 'FD', 141.21)
            ds.add_new([0x0018, 0x0084], 'DS', 63.887)
            
            ds.add_new([0x0018, 0x1060], 'DS', trigger_time)
            trigger_time += 32.0

            try:
                # delete header info
                del ds[0x0008, 0x2142]
                del ds[0x0008, 0x2143]
                del ds[0x0008, 0x2144]
                del ds[0x0018, 0x0040]
            except:
                pass

            # save dicom to new folder
            new_path = out_dir + '/{}/'.format(view) + file.split('\\')[-1]
            #print('Saving {}'.format(new_path))
            ds.save_as(new_path)

In [10]:
#### RUN THE SCRIPTS ####

for patient in os.listdir(IN_PATH + '/SA/'):
    print('Running for patient: {}'.format(patient))
    
    # run for short axis
    patient_path = IN_PATH + '/SA/' + patient
    
    print('Saving SA dicoms')
    ct2mri_SA(patient_path, OUT_PATH, frames=frames, size=size)
    
    # run for long axis
    patient_path = IN_PATH + '/LA/' + patient
    
    print('Saving LA dicoms')
    ct2mri_LA(patient_path, OUT_PATH, frames=frames, size=size)

    print('Done!')

Running for patient: Cvc1901261322
Saving SA dicoms
Saving LA dicoms
Done!
